In [2]:
### LOad the model
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
model = load_model('model.h5')

In [5]:
with open('gender_encoder.pkl','rb') as file:
    gender_encoder = pickle.load(file)
with open('ohe_encoder.pkl','rb') as file:
    ohe_encoder = pickle.load(file)
with open('scale.pkl','rb') as file:
    scalar = pickle.load(file)


In [6]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [8]:
data_df = pd.DataFrame.from_dict([input_data])
data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [10]:
numerical_cols = data_df.select_dtypes(include=np.number).columns
numerical_cols

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [11]:
data_df[numerical_cols]= scalar.transform(data_df[numerical_cols])

In [12]:
data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.535985,France,Male,0.104794,-0.695393,-0.257811,0.808436,0.649203,0.974817,-0.876832


In [13]:
data_df['Gender'] = gender_encoder.transform(data_df['Gender'])

In [14]:
X_encoded = ohe_encoder.transform(data_df[['Geography']].fillna('Unknown'))
X_encoded_df = pd.DataFrame(X_encoded, columns=ohe_encoder.get_feature_names_out(), index=data_df.index)
X_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [15]:
data_df = data_df.drop(columns=['Geography'])
data_df = pd.concat([data_df, X_encoded_df], axis=1)

In [16]:
data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,-0.535985,1,0.104794,-0.695393,-0.257811,0.808436,0.649203,0.974817,-0.876832,1.0,0.0,0.0


In [17]:
prediction=model.predict(data_df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


array([[0.02647716]], dtype=float32)

In [18]:
proba = prediction[0][0]

In [19]:
if(proba>0.5):
    print("customer is likely to churn")
else:
    print("customer is not likely to churn")

customer is not likely to churn
